In [17]:
from gen3.auth import Gen3Auth
#NOTE: using the Gen3File object to get presigned url so no need for these 
# (previously used to call directly from fence API)
#from urllib.parse import parse_qsl, urlencode, urlparse, urlunparse
from gen3.file import Gen3File
from gen3.index import Gen3Index
from gen3.submission import Gen3Submission
import os
import requests
import hashlib

In [18]:
COMMONS = 'https://jcoin.datacommons.io/'
CREDS_PATH = 'credentials.json'
AWS_BUCKET = "s3://jcoinprod-default-258867494168-upload"


In [19]:
program = "JCOIN"
project = "" #project as it appears in fence (ie profile)
uploader = "" #uploader identifier (ie email)

In [20]:
auth = Gen3Auth(COMMONS,CREDS_PATH)
index = Gen3Index(auth)
sub = Gen3Submission(auth)

## Prototype code for one file upload

Current gen3 file information 
(this includes current_object_id of core measures mapped to sheepdog and 
the indexed info of this sheepdog record)

In [21]:
for record in sub.export_node(program,project,"reference_file",fileformat='json')['data']:
    if f"core-measures" in record.get("submitter_id",""):
        current_record = record.copy()

[2022-11-20 16:50:01,414][WARNING] failed to write token cache file: C:\Users\kranz-michael/.cache/gen3/token_cache_e0c34db1a31432a1762ea4828d951154
[2022-11-20 16:50:01,415][WARNING] [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\kranz-michael/.cache/gen3/token_cache_e0c34db1a31432a1762ea4828d951154.tmp_eraseme_46717_1668984601' -> 'C:\\Users\\kranz-michael/.cache/gen3/token_cache_e0c34db1a31432a1762ea4828d951154'
[2022-11-20 16:50:01,416][WARNING] backoff: call gen3.auth._write_to_file(<gen3.auth.Gen3Auth object at 0x000001B3D96D41F0>, C:\Users\kranz-michael/.cache/gen3/token_cache_e0c34db1a31432a1762ea4828d951154, eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImZlbmNlX2tleV8yMDIwLTAxLTAyVDA2OjM1OjA4WiJ9.eyJwdXIiOiJhY2Nlc3MiLCJpc3MiOiJodHRwczovL2pjb2luLmRhdGFjb21tb25zLmlvL3VzZXIiLCJhdWQiOlsiaHR0cHM6Ly9qY29pbi5kYXRhY29tbW9ucy5pby91c2VyIiwiZmVuY2UiLCJnb29nbGVfc2VydmljZV9hY2NvdW50IiwiZ29vZ2xlX2NyZWRlbnRpYWxzIiwiYWRtaW4iLCJvcGVuaWQiLCJkYXRhIiwidXNlciIsImdvb2

In [22]:
current_object_id = current_record['object_id']
file_name = current_record['file_name']

current_index = index.get_record(current_object_id)

Local file information

In [23]:
file_path = f"core-measures/{file_name}"
with open(file_path, 'rb') as f:
    data = f.read()
    
# get file info
new_md5sum = hashlib.md5(data).hexdigest()
new_filesize = os.path.getsize(file_path)

In [24]:
#generate new version num
if current_index['version']:
    #make this a command (eg indicating which of the 3 version enhancements it is)
    pass
else:
    new_version_num = '.'.join(["0","0","1"]) #form of incremental version change code in above if statemnt 


Check if new version locally

In [25]:
#TODO: check if new version of file
is_new_version = new_md5sum!=current_index['hashes']['md5']

In [26]:
is_new_version

True

Mint new GUID

In [27]:
new_object_id = requests.get(f"{COMMONS}/index/guid/mint?count=1").json()["guids"][0]

Create new indexd record

In [28]:
# update new record more index metadata
#TODO: add version string
default_authz = '/programs/{}/projects/{}'.format(program, project)
authz= [default_authz]

new_index = index.create_new_version(
    guid=current_object_id,
    hashes={'md5':new_md5sum},
    size=new_filesize,
    did=new_object_id, 
    authz=authz
)

Upload new version of file

In [29]:
#get presigned url to upload
gen3files = Gen3File(auth)
new_file_presign = gen3files.upload_file_to_guid(new_object_id, file_name, expires_in=3600)
new_file_url = new_file_presign['url']
headers = {
    "Content-Type":"application/binary",
}
upload = requests.put(new_file_url, data=data, headers=headers)
upload.raise_for_status()

In [30]:
file_url = f"{AWS_BUCKET}/{new_object_id}/{file_name}"
index.update_record(
    new_object_id,
    urls=[file_url],
    # urls_metadata={file_url:{}}
)

{'acl': [],
 'authz': ['/programs/JCOIN/projects/NYU'],
 'baseid': 'c34ef61a-5369-44ae-ba22-84b5c7df25f1',
 'created_date': '2022-11-20T22:50:39.133153',
 'did': 'dg.6VTS/233fb7a3-f074-4451-8fa2-f3a658a860ed',
 'file_name': None,
 'form': 'object',
 'hashes': {'md5': '7954e74bb124ae401cb9d828306e4b2f'},
 'metadata': {},
 'rev': '7b4855d8',
 'size': 1066220,
 'updated_date': '2022-11-20T22:50:52.388955',
 'uploader': None,
 'urls': ['s3://jcoinprod-default-258867494168-upload/dg.6VTS/233fb7a3-f074-4451-8fa2-f3a658a860ed/core-measures-nyu.zip'],
 'urls_metadata': {'s3://jcoinprod-default-258867494168-upload/dg.6VTS/233fb7a3-f074-4451-8fa2-f3a658a860ed/core-measures-nyu.zip': {}},
 'version': None}

Map to sheepdog

In [31]:
  sheepdog_rec = {
    "core_metadata_collections": [
      {
        "submitter_id": f"core-measures-{project}"
      }
  ],
  "type": "reference_file",
  "project_id": f"{program}-{project}",
  "submitter_id": f"core-measures-{project}",
  "data_category": "Core Measures",
  "data_format": "ZIP",
  "data_type": "Interview",
  "md5sum":new_md5sum,
  "file_size":new_filesize,
  "file_name":file_name,
  "object_id":new_object_id,
}

In [32]:
sub.submit_record(program,project,json=sheepdog_rec)

[2022-11-20 16:50:55,168][   INFO] 
Using the Sheepdog API URL https://jcoin.datacommons.io/api/v0/submission/JCOIN/NYU



{'code': 200,
 'created_entity_count': 1,
 'entities': [{'action': 'create',
   'errors': [],
   'id': 'bca16800-0e59-4f40-baca-23262dc20a61',
   'type': 'reference_file',
   'unique_keys': [{'project_id': 'JCOIN-NYU',
     'submitter_id': 'core-measures-NYU'}],
   'valid': True,
   'warnings': []}],
 'entity_error_count': 0,
 'message': 'Transaction successful.',
 'success': True,
 'transaction_id': 9351,
 'transactional_error_count': 0,
 'transactional_errors': [],
 'updated_entity_count': 0}